# This notebook will be mainly used for the capstone project.

import all needed libs

In [2]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

'conda' is not recognized as an internal or external command,
operable program or batch file.
'conda' is not recognized as an internal or external command,
operable program or batch file.
Libraries imported.


In [3]:
address = 'Shanghai, CN'

geolocator = Nominatim(user_agent="sh_explore")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Shanghai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Shanghai are 31.2322758, 121.4692071.


In [4]:
CLIENT_ID = 'JAWBNTBS0BQFJG3NFDJSXJ2JHQ2WHLOT4HGWS55ACWLFYGBK' # your Foursquare ID
CLIENT_SECRET = 'S04CUSSD4H5MMCWQNFTBVDD4AQMISJRMPODX0QOFNZU2YFLK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [6]:
LIMIT = 300 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=JAWBNTBS0BQFJG3NFDJSXJ2JHQ2WHLOT4HGWS55ACWLFYGBK&client_secret=S04CUSSD4H5MMCWQNFTBVDD4AQMISJRMPODX0QOFNZU2YFLK&v=20180605&ll=31.2322758,121.4692071&radius=1000&limit=100'

In [55]:
results = requests.get(url).json()
results

u', '上海市', '中国']},
       'categories': [{'id': '52af3b593cf9994f4e043c00',
         'name': 'Shanghai Restaurant',
         'pluralName': 'Shanghai Restaurants',
         'shortName': 'Shanghai',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-4daac6dc0cb6a89c6274fa2f-83'},
     {'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f82cbe2e4b0d9965bf81c83',
       'name': 'Martini Bar At The Langham',
       'location': {'lat': 31.22359081800944,
        'lng': 121.46986774453954,
        'labeledLatLngs': [{'label': 'display',
          'lat': 31.22359081800944,
          'lng': 121.46986774453954}],
        'distance': 968,
        'cc': 'CN',
        'city': 'Huangpu',
        'state': '上海市',
   

In [56]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [57]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#print(nearby_venues.shape)
nearby_venues.head()

,name,categories,lat,lng
0,Shanghai Grand Theater (上海大剧院),Theater,31.231030,121.467263
1,JW Marriott Hotel Shanghai at Tomorrow Square ...,Hotel,31.232216,121.465260
2,People's Park (人民公园),Park,31.234009,121.467703
3,Shanghai Museum (上海博物馆),Art Museum,31.230427,121.470898
4,Shanghai Art Museum (上海美术馆),Art Museum,31.232784,121.466496


In [58]:
print('{} venues in {} categories were returned by Foursquare.'.format(nearby_venues.shape[0], len(nearby_venues['categories'].unique())))

100 venues in 54 categories were returned by Foursquare.


In [59]:
nearby_venues

,name,categories,lat,lng
0,Shanghai Grand Theater (上海大剧院),Theater,31.231030,121.467263
1,JW Marriott Hotel Shanghai at Tomorrow Square ...,Hotel,31.232216,121.465260
2,People's Park (人民公园),Park,31.234009,121.467703
3,Shanghai Museum (上海博物馆),Art Museum,31.230427,121.470898
4,Shanghai Art Museum (上海美术馆),Art Museum,31.232784,121.466496
5,Barbarossa,Lounge,31.233322,121.467179
6,The Yangtze Boutique Hotel (扬子精品酒店),Hotel,31.235331,121.472178
7,People's Square (人民广场),Plaza,31.234279,121.470520
8,Le Royal Club Lounge,Lounge,31.236404,121.471364
9,Jing'an Sculpture Park (静安雕塑公园),Sculpture Garden,31.234794,121.463911


In [60]:
top_categories = nearby_venues['categories'].value_counts().head(10)
top_categories

Hotel                   12
Coffee Shop              8
Café                     5
Chinese Restaurant       5
Fast Food Restaurant     3
Indian Restaurant        3
Park                     3
Art Museum               3
Lounge                   3
Hotel Bar                2
Name: categories, dtype: int64

In [61]:
#top_categories.index
top_venues = nearby_venues[nearby_venues['categories'].isin(top_categories.index)]
top_venues.reset_index(drop=True)

,name,categories,lat,lng
0,JW Marriott Hotel Shanghai at Tomorrow Square ...,Hotel,31.232216,121.465260
1,People's Park (人民公园),Park,31.234009,121.467703
2,Shanghai Museum (上海博物馆),Art Museum,31.230427,121.470898
3,Shanghai Art Museum (上海美术馆),Art Museum,31.232784,121.466496
4,Barbarossa,Lounge,31.233322,121.467179
5,The Yangtze Boutique Hotel (扬子精品酒店),Hotel,31.235331,121.472178
6,Le Royal Club Lounge,Lounge,31.236404,121.471364
7,Starbucks (星巴克),Coffee Shop,31.235657,121.467884
8,Le Royal Méridien Shanghai (上海世茂皇家艾美酒店),Hotel,31.236171,121.471357
9,Lobby Bar @ 38th floor - JW Marriott,Hotel Bar,31.232421,121.465553


In [62]:
top_venues.shape

(47, 4)

In [63]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the categories
colors_array = cm.rainbow(np.linspace(0, 1, len(top_categories)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
#markers_colors = []
for lat, lon, poi, category in zip(top_venues['lat'], top_venues['lng'], top_venues['name'], top_venues['categories']):
    label = folium.Popup(poi, parse_html=True)
    clist = list(top_categories.index)
    clr = rainbow[clist.index(category)]
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=clr,
        fill=True,
        fill_color=clr,
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [64]:
address = 'Beijing, CN'

geolocator = Nominatim(user_agent="sh_explore")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Beijing are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Shanghai are 39.906217, 116.3912757.


In [71]:
LIMIT = 300 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=JAWBNTBS0BQFJG3NFDJSXJ2JHQ2WHLOT4HGWS55ACWLFYGBK&client_secret=S04CUSSD4H5MMCWQNFTBVDD4AQMISJRMPODX0QOFNZU2YFLK&v=20180605&ll=39.906217,116.3912757&radius=5000&limit=300'

In [72]:
results = requests.get(url).json()
results

ormattedAddress': ['北京市', '北京市', '中国']},
       'categories': [{'id': '4bf58dd8d48988d145941735',
         'name': 'Chinese Restaurant',
         'pluralName': 'Chinese Restaurants',
         'shortName': 'Chinese',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-4e6385e1c65b2dc8a014f8b8-82'},
     {'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b91f135f964a52065e033e3',
       'name': 'Ritan Park (日坛公园)',
       'location': {'address': '6 Ritan N Rd',
        'crossStreet': 'Near Chaowai Ave',
        'lat': 39.91449278949437,
        'lng': 116.4380943775177,
        'labeledLatLngs': [{'label': 'display',
          'lat': 39.91449278949437,
          'lng': 116.4380943775177}],
        'dist

In [73]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [74]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#print(nearby_venues.shape)
nearby_venues

,name,categories,lat,lng
0,Tian'anmen Tower (天安门城楼),Historic Site,39.906562,116.391582
1,Zhongshan Park (Zhongshan Park 中山公园),Park,39.908765,116.388602
2,National Centre for the Performing Arts (国家大剧院),Performing Arts Venue,39.903936,116.383541
3,端门,Historic Site,39.908694,116.391192
4,Gate of Supreme Harmony (太和门),Historic Site,39.913719,116.390880
5,Lost Heaven 花马天堂,Yunnan Restaurant,39.900272,116.395612
6,Blue Note Beijing,Lounge,39.899744,116.395392
7,Din Tai Fung (鼎泰丰),Dumpling Restaurant,39.913630,116.405766
8,Hall of Supreme Harmony (太和殿),Historic Site,39.915466,116.390862
9,Forbidden City (Palace Museum) (故宫（博物院）),Historic Site,39.917702,116.390662


In [75]:
print('{} venues in {} categories were returned by Foursquare.'.format(nearby_venues.shape[0], len(nearby_venues['categories'].unique())))

100 venues in 42 categories were returned by Foursquare.


In [76]:
bj_top_categories = nearby_venues['categories'].value_counts().head(10)
bj_top_categories

Historic Site             18
Hotel                     11
Park                       7
Café                       6
Chinese Restaurant         4
Coffee Shop                4
Peking Duck Restaurant     3
Temple                     3
Shopping Mall              3
Yunnan Restaurant          3
Name: categories, dtype: int64

In [80]:
bj_top_venues = nearby_venues[nearby_venues['categories'].isin(bj_top_categories.index)]
bj_top_venues.reset_index(drop=True)

,name,categories,lat,lng
0,Tian'anmen Tower (天安门城楼),Historic Site,39.906562,116.391582
1,Zhongshan Park (Zhongshan Park 中山公园),Park,39.908765,116.388602
2,端门,Historic Site,39.908694,116.391192
3,Gate of Supreme Harmony (太和门),Historic Site,39.913719,116.390880
4,Lost Heaven 花马天堂,Yunnan Restaurant,39.900272,116.395612
5,Hall of Supreme Harmony (太和殿),Historic Site,39.915466,116.390862
6,Forbidden City (Palace Museum) (故宫（博物院）),Historic Site,39.917702,116.390662
7,Hall of Preserving Harmony (保和殿),Historic Site,39.916799,116.390809
8,Palace of Heavenly Purity (乾清宫),Historic Site,39.918491,116.390637
9,New World Beijing Hotel (北京新世界酒店),Hotel,39.897296,116.405923


In [86]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the categories
colors_array = cm.rainbow(np.linspace(0, 1, len(bj_top_categories)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
#markers_colors = []
for lat, lon, poi, category in zip(bj_top_venues['lat'], bj_top_venues['lng'], bj_top_venues['name'], bj_top_venues['categories']):
    label = folium.Popup(poi, parse_html=True)
    clist = list(bj_top_categories.index)
    clr = rainbow[clist.index(category)]
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=clr,
        fill=True,
        fill_color=clr,
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [88]:
address = 'Hongkong, CN'

geolocator = Nominatim(user_agent="sh_explore")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hongkong are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hongkong are 22.2793278, 114.1628131.


In [89]:
LIMIT = 300 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=JAWBNTBS0BQFJG3NFDJSXJ2JHQ2WHLOT4HGWS55ACWLFYGBK&client_secret=S04CUSSD4H5MMCWQNFTBVDD4AQMISJRMPODX0QOFNZU2YFLK&v=20180605&ll=22.2793278,114.1628131&radius=1000&limit=300'

In [90]:
results = requests.get(url).json()
results

34dd868e2',
       'name': 'Brass Spoon',
       'location': {'address': 'G/F, 10 Pottinger St',
        'lat': 22.2833287811399,
        'lng': 114.15609290015186,
        'labeledLatLngs': [{'label': 'display',
          'lat': 22.2833287811399,
          'lng': 114.15609290015186}],
        'distance': 823,
        'cc': 'HK',
        'city': '中環',
        'state': '中西區',
        'country': '香港',
        'formattedAddress': ['G/F, 10 Pottinger St', '中西區', '香港']},
       'categories': [{'id': '4bf58dd8d48988d142941735',
         'name': 'Asian Restaurant',
         'pluralName': 'Asian Restaurants',
         'shortName': 'Asian',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-589552aa561ded434dd868e2-82'},
     {'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
        

In [91]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#print(nearby_venues.shape)
nearby_venues

,name,categories,lat,lng
0,Hong Kong Park (香港公園),Park,22.277700,114.161854
1,Ruth's Chris Steak House (茹絲葵牛排餐廳),Steakhouse,22.279188,114.163833
2,Hong Kong Park Aviary (香港公園觀鳥園),Zoo,22.277140,114.161399
3,Pure Fitness,Gym / Fitness Center,22.278475,114.161363
4,The Murray Hong Kong (香港美利酒店),Hotel,22.278127,114.160392
5,Island Shangri-La (港島香格里拉大酒店),Hotel,22.277193,114.164310
6,Great Food Hall,Supermarket,22.277923,114.164712
7,Pacific Place (太古廣場),Shopping Mall,22.277618,114.165019
8,Howard's Gourmet (好酒好蔡),Chinese Restaurant,22.281469,114.161404
9,Sift Patisserie,Cupcake Shop,22.280922,114.159885


In [92]:
print('{} venues in {} categories were returned by Foursquare.'.format(nearby_venues.shape[0], len(nearby_venues['categories'].unique())))

100 venues in 52 categories were returned by Foursquare.


In [93]:
hk_top_categories = nearby_venues['categories'].value_counts().head(10)
hk_top_categories

Hotel                   9
Café                    6
Steakhouse              5
Italian Restaurant      5
Cantonese Restaurant    5
Yoga Studio             4
Bar                     4
Japanese Restaurant     4
Chinese Restaurant      3
Clothing Store          3
Name: categories, dtype: int64

In [95]:
hk_top_venues = nearby_venues[nearby_venues['categories'].isin(hk_top_categories.index)]
hk_top_venues.reset_index(drop=True)

,name,categories,lat,lng
0,Ruth's Chris Steak House (茹絲葵牛排餐廳),Steakhouse,22.279188,114.163833
1,The Murray Hong Kong (香港美利酒店),Hotel,22.278127,114.160392
2,Island Shangri-La (港島香格里拉大酒店),Hotel,22.277193,114.164310
3,Howard's Gourmet (好酒好蔡),Chinese Restaurant,22.281469,114.161404
4,The Upper House (奕居),Hotel,22.277499,114.166323
5,Pure Yoga,Yoga Studio,22.278106,114.164754
6,Lawry's The Prime Rib,Steakhouse,22.280859,114.162527
7,Café Gray Deluxe,Café,22.277506,114.166320
8,Mandarin Grill + Bar (文華扒房＋酒吧),Steakhouse,22.281928,114.159408
9,Mandarin Oriental Hong Kong (香港文華東方酒店),Hotel,22.281857,114.159382


In [99]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the categories
colors_array = cm.rainbow(np.linspace(0, 1, len(hk_top_categories)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
#markers_colors = []
for lat, lon, poi, category in zip(hk_top_venues['lat'], hk_top_venues['lng'], hk_top_venues['name'], hk_top_venues['categories']):
    label = folium.Popup(poi, parse_html=True)
    clist = list(hk_top_categories.index)
    clr = rainbow[clist.index(category)]
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=clr,
        fill=True,
        fill_color=clr,
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [102]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(len(top_categories)):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = ['Shanghai', 'Beijing', 'Hongkong']

city_venues_sorted.iloc[0, 1:] = list(top_categories.index)
city_venues_sorted.iloc[1, 1:] = list(bj_top_categories.index)
city_venues_sorted.iloc[2, 1:] = list(hk_top_categories.index)

city_venues_sorted

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Shanghai,Hotel,Coffee Shop,Café,Chinese Restaurant,Fast Food Restaurant,Indian Restaurant,Park,Art Museum,Lounge,Hotel Bar
1,Beijing,Historic Site,Hotel,Park,Café,Chinese Restaurant,Coffee Shop,Peking Duck Restaurant,Temple,Shopping Mall,Yunnan Restaurant
2,Hongkong,Hotel,Café,Steakhouse,Italian Restaurant,Cantonese Restaurant,Yoga Studio,Bar,Japanese Restaurant,Chinese Restaurant,Clothing Store
